<h5>Imports be here<h5>

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pathlib import Path

<p>Reading the document<p>

In [5]:
#Open Excel files with different extensions
filename = 'train.csv'
file_path = Path(filename)
file_extension = file_path.suffix.lower()[1:]

if file_extension == 'xlsx':
    data = pd.read_excel(file_path, engine='openpyxl')
elif file_extension == 'xls':
    data = pd.read_excel(file_path)
elif file_extension == 'csv':
    data = pd.read_csv(file_path)
else:
    raise Exception("File not supported")


In [6]:
data.head()

,Cust_ID;Churn;Name;Longevity;Year_Birth;TypeTravel;RoomType;RewardPoints;Comfort;ReceptionSchedule;FoodDrink;Location;Wifi;Amenities;Staff;OnlineBooking;PriceQuality;RoomSpace;CheckOut;Checkin;Cleanliness;BarService
0,1;churn;Ms. Nicole Clarke;yes;1974;business;si...
1,2;nochurn;Mr. Jesse Scott;yes;1965;business;si...
2,3;churn;Mr. Christopher Carter;yes;1973;busine...
3,4;nochurn;Ms. Jennifer Morgan;yes;1993;leisure...
4,5;nochurn;Mr. Michael White;yes;1989;business;...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15589 entries, 0 to 15588
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                   Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                   --------------  ----- 
 0   Cust_ID;Churn;Name;Longevity;Year_Birth;TypeTravel;RoomType;RewardPoints;Comfort;ReceptionSchedule;FoodDrink;Location;Wifi;Amenities;Staff;OnlineBooking;PriceQuality;RoomSpace;CheckOut;Checkin;Cleanliness;BarService  15589 non-null  object
dtypes: object(1)
memory usage: 121.9+ KB


In [8]:
data.shape

(15589, 1)

In [9]:
data2 = data.copy()